<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 2*

# Document Representations: Bag-Of-Words

In [4]:
import re
import string

#!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /Users/TomasFox/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/TomasFox/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

If you choose to not to scrape the data, there is a CSV with outdated data in the directory. Remeber, if you scrape Indeed, you're helping yourself find a job. ;)

In [6]:
!python -m spacy validate
!python -m spacy download en_core_web_lg

✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.4) ======================
ℹ spaCy installation: /anaconda3/lib/python3.7/site-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.1.0   ✔
package   en-core-web-lg   en_core_web_lg   2.0.0   --> 2.1.0     
link      en               en_core_web_sm   2.1.0   ✔


============================== Install updates ==============================
Use the following commands to update the model packages:
python -m spacy download en_core_web_lg

    100% |████████████████████████████████| 826.9MB 18.6MB/s ta 0:00:01  1% |▌                               | 13.6MB 1.6MB/s eta 0:08:39    1% |▋                               | 14.9MB 977kB/s eta 0:13:51    2% |▊                               | 17.3MB 3.1MB/s eta 0:04:25    2% |█                               | 23.1MB 1.1MB/s eta 0:12:05    2% |█                               | 24.4MB 9

## 2) Use Spacy to tokenize / clean the listings 

In [5]:
df = pd.read_csv('job_listings.csv')
df.head()

,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


In [6]:
text = df['description']
text.head()

0    b"<div><div>Job Requirements:</div><ul><li><p>...
1    b'<div>Job Description<br/>\n<br/>\n<p>As a Da...
2    b'<div><p>As a Data Scientist you will be work...
3    b'<div class="jobsearch-JobMetadataHeader icl-...
4    b'<ul><li>Location: USA \xe2\x80\x93 multiple ...
Name: description, dtype: object

In [7]:
def tokenize(text):
    """Parses a string into a list of semantic units (words)

    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    # Ned's Solution:
    sample = re.sub(r'[^a-zA-Z ^0-9]', '', text)
    tokens = sample.lower().split()
    
    return tokens

In [8]:
df['tokens'] = text.apply(tokenize)
df['tokens'].head()

0    [bdivdivjob, requirementsdivullipnconceptual, ...
1    [bdivjob, descriptionbrnbrnpas, a, data, scien...
2    [bdivpas, a, data, scientist, you, will, be, w...
3    [bdiv, classjobsearchjobmetadataheader, icluxs...
4    [bullilocation, usa, xe2x80x93, multiple, loca...
Name: tokens, dtype: object

In [22]:
#let's use spacy 

import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load('en_core_web_lg')

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [23]:
tokens = []

for review in tokenizer.pipe(text):
    doc_tokens = [token.text for token in review]
    tokens.append(doc_tokens)
    
df['tokens'] = tokens

# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [50]:
from sklearn.feature_extraction.text import CountVectorizer 

vectorizer = CountVectorizer(stop_words='english')

vectorizer.fit(text)

vectorizer.vocabulary_

dtm = vectorizer.transform(text)
dtm.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 2, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]])

In [51]:
dtm = pd.DataFrame(dtm.todense(), columns = vectorizer.get_feature_names())

In [67]:
dtm.head()

,00,000,02115,03,0356,04,057,062,06366,08,...,zero,zeus,zf,zheng,zillow,zogsports,zones,zoom,zuckerberg,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
dtm.sum().sort_values(ascending=False).head(10)

li            12785
div            6870
data           4452
br             3894
ul             2636
experience     1584
xe2            1417
x80            1404
business       1210
work           1077
dtype: int64

# 4) Visualize the most common word counts

In [77]:
tokens = []

""" Update those tokens """
for doc in tokenizer.pipe(df['description'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc: 
        if (token.is_stop == False) and (token.is_punct == False):
            doc_tokens.append(token.text.lower())
            
    tokens.append(doc_tokens)
    
df['tokens'] = tokens

In [78]:
# Object from Base Python
from collections import Counter

# The object `Counter` takes an iterable, but you can instantiate an empty one and update it. 
word_counts = Counter()


# Update it based on a split of each of our documents
df['tokens'].apply(lambda x: word_counts.update(x))


# Print out the 10 most common words
word_counts.most_common(10)

[('data', 3748),
 ('business', 1008),
 ('experience', 940),
 ('work', 875),
 ('team', 726),
 ('machine', 660),
 ('learning', 640),
 ('science', 564),
 ('statistical', 530),
 ('new', 517)]

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

dtm = tfidf.fit_transform(df['description'])

docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
docs.head()

,00,000,04,10,100,1079302,11,12,125,14,...,yes,yeti,york,young,yrs,zenreach,zeus,zf,zillow,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


 # 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [84]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [85]:
nn.kneighbors(dtm.todense()[0])

(array([[0.        , 0.86248789, 0.87371026, 0.88181475, 0.88777735]]),
 array([[  0, 276, 301, 264,  71]]))

In [86]:
ideal = ["""Data scientists utilize their analytical, statistical,
and programming skills to collect, analyze, and interpret large data sets. 
They then use this information to develop data-driven solutions to 
difficult business challenges. Data scientists commonly 
have a bachelor's degree in statistics, math, computer science, 
or economics. Data scientists have a wide range of technical
competencies including: statistics and machine learning, coding
languages, databases, machine learning, and reporting technologies."""]

In [87]:
new = tfidf.transform(ideal)
nn.kneighbors(new.todense())

(array([[1.27520712, 1.28386618, 1.28605981, 1.28728691, 1.28728691]]),
 array([[261, 327, 176, 184, 147]]))

In [82]:
df['description'][261]

"b'<div><p>The Data Science Engineer, Mintel Futures is a core part of Mintel\\xe2\\x80\\x99s data science team that will have the opportunity to work on a wide array of initiatives across varying aspects of Mintel\\xe2\\x80\\x99s business and data. This individual will help manage the full analytics lifecycle of advanced projects; help to identify new, impactful ways to apply machine learning to Mintel\\xe2\\x80\\x99s data; work alongside data scientists and business stakeholders to implement solutions that provide valuable insights; and aide in the design and development of a modern data analytics environment.</p>\\n<p><b>\\nWhat You Will Do:</b></p>\\n<ul>\\n<li>Play an integral role in shaping the underlying technology environment for Mintel\\xe2\\x80\\x99s fast growing team of data scientists and data analysts</li>\\n<li>Assist in the acquisition and management of a variety of data sources for large-scale analysis</li>\\n<li>Identify opportunities for predictive modeling or other 

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.